The goal of this notebook is to find strategies that are able to find resolving sets as small as possible. We start with the Barabàsi-Albert model and then simulate the strategies on real graphs.

In [4]:
import networkx as nx
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
import sys
sys.path.append('../')
from helpers import *
import plotly.graph_objs as go
%load_ext autoreload
%autoreload 2

## Barabási–Albert model

In [5]:
# By default, we will use the following parameters
# n = 500
# m = 30
# nb_graph = 1
# nb_of_iters = 100

# can be used to speed up simulations as we know the theoritical value
# lower_bound = 60
# upper_bound = 160

In [76]:
def sim_centrality(centrality_f, lower_bound=20, upper_bound=110, n=500, m=30, nb_graph=1, nb_of_iters=100, seed=0):
    solutions_prob_c = {}
    random.seed(seed)
    for _ in range(nb_graph):
        # Generate the random graph and compute shortest paths
        G = nx.barabasi_albert_graph(n, m)
        length = dict(nx.all_pairs_shortest_path_length(G))

        c = centrality_f(G)

        # normalize the betweenness centralities
        c_values = np.array(list(c.values()))
        c_norm = c_values / sum(c_values)
        node_list = list(c.keys())

        for nb in range(0, lower_bound):
            solutions_prob_c[nb] = solutions_prob_c.get(nb, 0) + 0  
        for nb in tqdm(range(lower_bound, upper_bound)): # G.number_of_nodes()
            num_nodes = nb # Number of nodes to sample
            count = 0
            for i in range(nb_of_iters):
                nodes = set(np.random.choice(node_list, p=c_norm, size=num_nodes, replace=False)) # Random set of nodes to test
                if is_resolving_set(G, nodes, length):
                    count += 1
            solutions_prob_c[nb] =  solutions_prob_c.get(nb, 0) + (count / nb_of_iters)
        for nb in range(upper_bound, n):
                solutions_prob_c[nb] = solutions_prob_c.get(nb, 0) + 1

    for i in range(n):
        solutions_prob_c[i] = solutions_prob_c[i] / nb_graph
        
    return solutions_prob_c

In [77]:
solutions_prob_bc_2 = sim_centrality(nx.betweenness_centrality, lower_bound=20, upper_bound=80, nb_graph=10)

100%|███████████████████████████████████████████| 60/60 [00:54<00:00,  1.11it/s]


In [151]:
solutions_prob_eigen = sim_centrality(nx.eigenvector_centrality, lower_bound=30, upper_bound=90, nb_graph=10)

100%|███████████████████████████████████████████| 60/60 [00:59<00:00,  1.00it/s]


In [152]:
solutions_prob_dc = sim_centrality(nx.degree_centrality, lower_bound=30, upper_bound=90, nb_graph=10)

100%|███████████████████████████████████████████| 60/60 [00:59<00:00,  1.01it/s]


In [153]:
solutions_prob_cc = sim_centrality(nx.closeness_centrality, lower_bound=20, upper_bound=100, nb_graph=10)

100%|███████████████████████████████████████████| 80/80 [01:21<00:00,  1.01s/it]


In [154]:
solutions_prob_flow_bc = sim_centrality(nx.current_flow_betweenness_centrality, lower_bound=30, upper_bound=90, nb_graph=10)

<class 'networkx.utils.decorators.argmap'> compilation 36:4: FutureWarning:

laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.

100%|███████████████████████████████████████████| 60/60 [00:59<00:00,  1.00it/s]


In [155]:
solutions_prob_so = sim_centrality(nx.second_order_centrality, lower_bound=25, upper_bound=90, nb_graph=10)

100%|███████████████████████████████████████████| 65/65 [01:03<00:00,  1.02it/s]


In [156]:
# Random strategy

n = 500
m = 30
nb_graph = 10
s = {}
nb_of_iters = 100

# can be used to speed up simulations as we know the theoritical value
lower_bound = 20
upper_bound = 100

solutions_rand = {}
random.seed(0)
for _ in range(nb_graph):
    # Generate the random graph and compute shortest paths
    G = nx.barabasi_albert_graph(n, m)
    length = dict(nx.all_pairs_shortest_path_length(G))
    
    for nb in range(0, lower_bound):
        solutions_rand[nb] = solutions_rand.get(nb, 0) + 0  
    for nb in tqdm(range(lower_bound, upper_bound)): # G.number_of_nodes()
        num_nodes = nb # Number of nodes to sample
        node_list = list(G.nodes())
        count = 0
        for i in range(nb_of_iters):
            nodes = set(random.sample(node_list, num_nodes)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                count += 1
        solutions_rand[nb] =  solutions_rand.get(nb, 0) + (count / nb_of_iters)
    for nb in range(upper_bound, n):
            solutions_rand[nb] = solutions_rand.get(nb, 0) + 1

for i in range(n):
    solutions_rand[i] = solutions_rand[i] / nb_graph

100%|███████████████████████████████████████████| 80/80 [00:36<00:00,  2.22it/s]


In [213]:
# Strategy where we draw only from first quntile (degree)

n = 500
m = 30
nb_of_iters = 100
nb_graphs = 10

solutions_first_q = {}

lower_bound = 20
upper_bound = 90

number_of_quintile = 5

for _ in range(nb_graphs):
    G = nx.barabasi_albert_graph(n, m)

    node_list = list(G.nodes())

    degree_list = [(n, d) for n, d in G.degree()]
    degree_list.sort(key=lambda x: x[1], reverse=True)
    num_vertices = len(degree_list)
    num_vertices_per_decile = n // number_of_quintile
    decile_num = 1
    decile_vertices = []

    for i in range(n):
        vertex = degree_list[i][0]
        degree = degree_list[i][1]
        decile_vertices.append(vertex)
        if (i + 1) % num_vertices_per_decile == 0:
            nx.set_node_attributes(G, {v: decile_num for v in decile_vertices}, 'decile')
            decile_num += 1
            decile_vertices = []

    length = dict(nx.all_pairs_shortest_path_length(G))

    quintile = 1      
    nodes_in_decile = [node for node, decile in nx.get_node_attributes(G, 'decile').items() if decile == quintile]
    
    
    for nb in range(0, lower_bound):
        solutions_first_q[nb] = solutions_first_q.get(nb, 0) + 0  
    for nb in tqdm(range(lower_bound, upper_bound)): # G.number_of_nodes()
        num_nodes = nb # Number of nodes to sample
        node_list = list(G.nodes())
        count = 0
        for i in range(nb_of_iters):
            nodes = set(random.sample(nodes_in_decile, num_nodes)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                count += 1
        solutions_first_q[nb] =  solutions_first_q.get(nb, 0) + (count / nb_of_iters)
    for nb in range(upper_bound, n):
            solutions_first_q[nb] = solutions_first_q.get(nb, 0) + 1
    
for i in range(n):
    solutions_first_q[i] = solutions_first_q[i] / nb_graphs

100%|███████████████████████████████████████████| 70/70 [00:27<00:00,  2.59it/s]


In [177]:
for i in range(n):
    solutions_first_q[i] = solutions_first_q[i] / nb_graphs

In [187]:
#with open('simulations/transition_eigen_centrality.pickle', 'wb') as file:
    #pickle.dump(solutions_prob_eigen, file, protocol=pickle.HIGHEST_PROTOCOL)

In [72]:
#Can be used to directyl import the results of the simulations


#with open('simulations/Barbasi-Albert/transition_random.pickle', "rb") as file:
    #solutions_rand = pickle.load(file)
#with open('simulations/Barbasi-Albert/transition_betweenness_centrality.pickle', "rb") as file:
    #solutions_prob_bc = pickle.load(file)
#with open('simulations/Barbasi-Albert/transition_degree_centrality.pickle', "rb") as file:
    #solutions_prob_dc = pickle.load(file)
#with open('simulations/Barbasi-Albert/transition_so_centrality.pickle', "rb") as file:
    #solutions_prob_so = pickle.load(file)
#with open('simulations/Barbasi-Albert/transition_closness_centrality.pickle', "rb") as file:
    #solutions_prob_cc = pickle.load(file)
#with open('simulations/Barbasi-Albert/transition_eigen_centrality.pickle', "rb") as file:
    #solutions_prob_eigen = pickle.load(file)

In [1]:
# Define your data
x = list(solutions_rand.keys())
y = list(solutions_rand.values())

x1 = list(solutions_first_q.keys())
y1 = list(solutions_first_q.values())

x3 = list(solutions_prob_bc.keys())
y3 = list(solutions_prob_bc.values())

x4 = list(solutions_prob_dc.keys())
y4 = list(solutions_prob_dc.values())

x5 = list(solutions_prob_so.keys())
y5 = list(solutions_prob_so.values())

x6 = list(solutions_prob_cc.keys())
y6 = list(solutions_prob_cc.values())

x8 = list(solutions_prob_eigen.keys())
y8 = list(solutions_prob_eigen.values())

# Define the trace for the scatter plot
trace = go.Scatter(x=x, y=y, mode='markers+lines', name='Random nodes')
trace1 = go.Scatter(x=x1, y=y1, mode='markers+lines', name='Nodes with high degree (top 20%)')
trace3 = go.Scatter(x=x3, y=y3, mode='markers+lines', name='Betweenness centrality')
trace4 = go.Scatter(x=x4, y=y4, mode='markers+lines', name='Degree centrality')
trace5 = go.Scatter(x=x5, y=y5, mode='markers+lines', name='Second order centrality')
trace6 = go.Scatter(x=x6, y=y6, mode='markers+lines', name='Closeness centrality')
trace8 = go.Scatter(x=x8, y=y8, mode='markers+lines', name='Eigenvector centrality')


# Define the layout
layout = go.Layout(#title='Probability of resolving the graph as a function of the subset cardinality', 
                   #title_x=0.5,
                   xaxis=dict(title='Cardinality of the subset'), 
                   yaxis=dict(title='Probability of resolving the graph'),
                   legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace, trace1, trace3, trace4, trace5, trace6, trace8], layout=layout)

# Show the figure
fig.show()

NameError: name 'solutions_rand' is not defined

In [223]:
def sim_centrality_box(centrality_f, n=500, m=30, nb_graph=15, nb_of_iters=100, seed=0):
    
    box_sol = []
    random.seed(seed)
    for _ in tqdm(range(nb_graph)):
    
        # Generate the random graph and compute shortest paths
        G = nx.barabasi_albert_graph(n, m)
        length = dict(nx.all_pairs_shortest_path_length(G))

        c = centrality_f(G)

        # normalize the betweenness centralities
        c_values = np.array(list(c.values()))
        c_norm = c_values / sum(c_values)
        node_list = list(c.keys())
        for _ in range(nb_of_iters):
            nodes = np.random.choice(node_list, p=c_norm, size=len(G), replace=False) # Random set of nodes to test
            for nb in range(0, len(G) + 1):
                #print(nb)
                if is_resolving_set(G, nodes[:nb], length):
                    box_sol.append(nb)
                    break
    return box_sol

In [159]:
# Random

n = 500
m = 30
nb_graph = 15
nb_of_iters = 100

rand_box_sol = []
random.seed(0)
for _ in range(nb_graph):

    # Generate the random graph and compute shortest paths
    G = nx.barabasi_albert_graph(n, m)
    length = dict(nx.all_pairs_shortest_path_length(G))

    for _ in tqdm(range(nb_of_iters)):
        nodes = list(random.sample(list(G.nodes), n))
        for nb in range(0, len(G)):
            if is_resolving_set(G, nodes[:nb], length):
                rand_box_sol.append(nb)
                break

100%|█████████████████████████████████████████| 100/100 [00:15<00:00,  6.60it/s]


In [160]:
dc_box_sol = sim_centrality_box(nx.degree_centrality)

100%|███████████████████████████████████████████| 15/15 [06:50<00:00, 27.38s/it]


In [161]:
bc_box_sol = sim_centrality_box(nx.betweenness_centrality)

100%|███████████████████████████████████████████| 15/15 [05:02<00:00, 20.18s/it]


In [162]:
cc_box_sol = sim_centrality_box(nx.closeness_centrality)

100%|███████████████████████████████████████████| 15/15 [10:17<00:00, 41.19s/it]


In [163]:
so_box_sol = sim_centrality_box(nx.second_order_centrality)

100%|███████████████████████████████████████████| 15/15 [16:32<00:00, 66.14s/it]


In [224]:
eigen_box_sol = sim_centrality_box(nx.eigenvector_centrality)

  0%|                                                    | 0/15 [00:00<?, ?it/s]

start


  7%|██▉                                         | 1/15 [00:31<07:16, 31.17s/it]

start


 13%|█████▊                                      | 2/15 [01:03<06:55, 31.98s/it]

...

 87%|█████████████████████████████████████▎     | 13/15 [06:44<01:03, 31.68s/it]

start


 93%|████████████████████████████████████████▏  | 14/15 [07:16<00:31, 31.88s/it]

start


100%|███████████████████████████████████████████| 15/15 [07:49<00:00, 31.27s/it]


In [164]:
dict_result = {'Betweenness centrality': bc_box_sol, 'Degree centrality': dc_box_sol, 'Closeness centrality': cc_box_sol , 'Random': rand_box_sol, 'Second order centrality': so_box_sol, }

In [83]:
#with open('simulations/Barbasi-Albert/centrality_boxes_15graph_100iters.pickle', "rb") as file:
    #dict_result = pickle.load(file)

In [84]:
#with open('simulations/centrality_boxes_15graph_100iters.pickle', 'wb') as file:
    #pickle.dump(dict_result, file, protocol=pickle.HIGHEST_PROTOCOL)

In [232]:
fig = go.Figure()

for group, values in dict_result2.items():
    fig.add_trace(go.Box(y=values, name=group))

fig.update_layout(
    yaxis=dict(
        title="Size of the resolving set",
        titlefont=dict(size=12, color='black')
    )
)       
    
fig.show()

## Copenhagen fb_friends graph

In [7]:
# read the file and create a graph
G_fb = nx.read_edgelist('../Real graphs simulations/Copenhagen graphs/fb_friends.csv/edges.csv', delimiter=',')

# print the number of nodes and edges
print('Number of nodes:', G_fb.number_of_nodes())
print('Number of edges:', G_fb.number_of_edges())

Number of nodes: 800
Number of edges: 6429


In [236]:
def sim_centrality_real_g(centrality_f, lower_bound=20, upper_bound=800, nb_of_iters=400, seed=0, step=10):
    solutions_prob_c = {}
    random.seed(seed)
    for _ in range(nb_graph):
        # Generate the random graph and compute shortest paths
        G = G_fb
        length = dict(nx.all_pairs_shortest_path_length(G))

        c = centrality_f(G)

        # normalize the betweenness centralities
        c_values = np.array(list(c.values()))
        c_norm = c_values / sum(c_values)
        node_list = list(c.keys())

        for nb in range(0, lower_bound):
            solutions_prob_c[nb] = solutions_prob_c.get(nb, 0) + 0  
        for nb in tqdm(range(lower_bound, upper_bound+1, step)): # G.number_of_nodes()
            num_nodes = nb # Number of nodes to sample
            count = 0
            for i in range(nb_of_iters):
                nodes = set(np.random.choice(node_list, p=c_norm, size=num_nodes, replace=False)) # Random set of nodes to test
                if is_resolving_set(G, nodes, length):
                    count += 1
            solutions_prob_c[nb] =  solutions_prob_c.get(nb, 0) + (count / nb_of_iters)
        for nb in range(upper_bound+1, G_fb.number_of_nodes()+1):
                solutions_prob_c[nb] = solutions_prob_c.get(nb, 0) + 1
        
    return solutions_prob_c

In [237]:
solutions_prob_eigen_fb = sim_centrality_real_g(nx.eigenvector_centrality)

100%|███████████████████████████████████████████| 79/79 [35:17<00:00, 26.80s/it]


In [238]:
solutions_prob_dc_fb = sim_centrality_real_g(nx.degree_centrality)

100%|███████████████████████████████████████████| 79/79 [33:01<00:00, 25.08s/it]


In [239]:
solutions_prob_cc_fb = sim_centrality_real_g(nx.closeness_centrality)

100%|███████████████████████████████████████████| 79/79 [33:01<00:00, 25.08s/it]


In [240]:
solutions_prob_so_fb = sim_centrality_real_g(nx.second_order_centrality)

100%|███████████████████████████████████████████| 79/79 [33:11<00:00, 25.21s/it]


In [241]:
# Random strategy

nb_graph = 1
s = {}
nb_of_iters = 800
step = 10

# can be used to speed up simulations
lower_bound = 20
upper_bound = 800

solutions_rand_fb = {}
random.seed(0)
for _ in range(nb_graph):
    # Generate the random graph and compute shortest paths
    G = G_fb
    length = dict(nx.all_pairs_shortest_path_length(G))
    
    for nb in range(0, lower_bound):
        solutions_rand_fb[nb] = solutions_rand_fb.get(nb, 0) + 0  
    for nb in tqdm(range(lower_bound, upper_bound+1, step)):
        num_nodes = nb # Number of nodes to sample
        node_list = list(G.nodes())
        count = 0
        for i in range(nb_of_iters):
            nodes = set(random.sample(node_list, num_nodes)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                count += 1
        solutions_rand_fb[nb] =  solutions_rand_fb.get(nb, 0) + (count / nb_of_iters)
    for nb in range(upper_bound+1, G_fb.number_of_nodes()+1):
            solutions_rand_fb[nb] = solutions_rand_fb.get(nb, 0) + 1

100%|█████████████████████████████████████████| 79/79 [1:00:12<00:00, 45.73s/it]


In [242]:
# Betweenness centrality

# can be used to speed up simulations
lower_bound = 20
upper_bound = 800
nb_of_iters = 800
step = 10
solutions_prob_bc_fb = {}
for _ in range(nb_graph):
    # Generate the random graph and compute shortest paths
    G = G_fb
    length = dict(nx.all_pairs_shortest_path_length(G))

    c = nx.betweenness_centrality(G, endpoints=True)

    # normalize the betweenness centralities
    c_values = np.array(list(c.values()))
    c_norm = c_values / sum(c_values)
    node_list = list(c.keys())

    for nb in range(0, lower_bound):
        solutions_prob_bc_fb[nb] = solutions_prob_bc_fb.get(nb, 0) + 0  
    for nb in tqdm(range(lower_bound, upper_bound+1, step)): # G.number_of_nodes()
        num_nodes = nb # Number of nodes to sample
        count = 0
        for i in range(nb_of_iters):
            nodes = set(np.random.choice(node_list, p=c_norm, size=num_nodes, replace=False)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                count += 1
        solutions_prob_bc_fb[nb] = solutions_prob_bc_fb.get(nb, 0) + (count / nb_of_iters)
    for nb in range(upper_bound+1, G_fb.number_of_nodes()+1):
            solutions_prob_bc_fb[nb] = solutions_prob_bc_fb.get(nb, 0) + 1


100%|█████████████████████████████████████████| 79/79 [1:06:28<00:00, 50.48s/it]


In [23]:
# Boost the proba of low degree nodes

lower_bound = 20
upper_bound = 40

solutions_prob_ms_d = {}
for _ in range(nb_graph):
    # Generate the random graph and compute shortest paths
    G = G_fb
    length = dict(nx.all_pairs_shortest_path_length(G))

    c = nx.degree_centrality(G)
    a = list(set(c.values()))
    a.sort()
    c = {key: 1000*max(c.values()) if value in a[:2] else value for key, value in c.items()}
    # normalize the betweenness centralities
    c_values = np.array(list(c.values()))
    c_norm = c_values / sum(c_values)
    node_list = list(c.keys())

    for nb in range(0, lower_bound):
        solutions_prob_ms_d[nb] = solutions_prob_ms_d.get(nb, 0) + 0  
    for nb in tqdm(range(lower_bound, upper_bound, 1)): # G.number_of_nodes()
        num_nodes = nb # Number of nodes to sample
        count = 0
        for i in range(nb_of_iters):
            nodes = set(np.random.choice(node_list, p=c_norm, size=num_nodes, replace=False)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                count += 1
        solutions_prob_ms_d[nb] =  solutions_prob_ms_d.get(nb, 0) + (count / nb_of_iters)
    for nb in range(upper_bound, G_fb.number_of_nodes()):
            solutions_prob_ms_d[nb] = solutions_prob_ms_d.get(nb, 0) + 1


100%|███████████████████████████████████████████| 20/20 [00:44<00:00,  2.22s/it]


In [24]:
# Boost the proba of low betweenness nodes

lower_bound = 20
upper_bound = 40

solutions_prob_ms_bc = {}
for _ in range(nb_graph):
    # Generate the random graph and compute shortest paths
    G = G_fb
    length = dict(nx.all_pairs_shortest_path_length(G))

    c = nx.betweenness_centrality(G)
    a = list(set(c.values()))
    a.sort()
    #c = {key: -(v-max(c.values())-min(c.values())) for key, v in c.items()}
    c = {key: 1000*max(c.values()) if value in a[:1] else value for key, value in c.items()}
    # normalize the betweenness centralities
    c_values = np.array(list(c.values()))
    c_norm = c_values / sum(c_values)
    node_list = list(c.keys())

    for nb in range(0, lower_bound):
        solutions_prob_ms_bc[nb] = solutions_prob_ms_bc.get(nb, 0) + 0  
    for nb in tqdm(range(lower_bound, upper_bound, 1)): # G.number_of_nodes()
        num_nodes = nb # Number of nodes to sample
        count = 0
        for i in range(nb_of_iters):
            nodes = set(np.random.choice(node_list, p=c_norm, size=num_nodes, replace=False)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                count += 1
        solutions_prob_ms_bc[nb] =  solutions_prob_ms_bc.get(nb, 0) + (count / nb_of_iters)
    for nb in range(upper_bound, G_fb.number_of_nodes()):
            solutions_prob_ms_bc[nb] = solutions_prob_ms_bc.get(nb, 0) + 1


100%|███████████████████████████████████████████| 20/20 [00:44<00:00,  2.23s/it]


In [37]:
# Strategy 1 with degree

lower_bound = 20
upper_bound = 40

solutions_prob_ms_bc_add = {}
for _ in range(nb_graph):
    # Generate the random graph and compute shortest paths
    G = G_fb

    nb_of_nodes_in_resolving_set = 20

    length = dict(nx.all_pairs_shortest_path_length(G))

    c = nx.degree_centrality(G)

    intensities = {str(key): 0 for key in G.nodes}

    # normalize the betweenness centralities
    c_values = np.array(list(c.values()))
    c_norm = c_values / sum(c_values)
    node_list = list(c.keys())
    
    for i in range(100):
        # Random set of nodes to test
        nodes = set(np.random.choice(node_list, p=c_norm, size=nb_of_nodes_in_resolving_set, replace=False))
        resolved = set_resolved(G, nodes, length)
        for node in resolved:
            intensities[node] += +1
    
    small = [key for key, val in intensities.items() if val < 40]
    nodes_to_add = get_nodes_with_diff_neighbors(G, small)
    
    for nb in range(0, lower_bound):
        solutions_prob_ms_bc_add[nb] = solutions_prob_ms_bc_add.get(nb, 0) + 0  
    for nb in tqdm(range(lower_bound, upper_bound, 1)): # G.number_of_nodes()
        num_nodes = nb - len(nodes_to_add) # Number of nodes to sample
        count = 0
        for i in range(nb_of_iters):
            nodes = nodes_to_add + list(np.random.choice(node_list, p=c_norm, size=num_nodes, replace=False)) # Random set of nodes to test
            if is_resolving_set(G, nodes, length):
                count += 1
        solutions_prob_ms_bc_add[nb] =  solutions_prob_ms_bc_add.get(nb, 0) + (count / nb_of_iters)
    for nb in range(upper_bound, G_fb.number_of_nodes()):
            solutions_prob_ms_bc_add[nb] = solutions_prob_ms_bc_add.get(nb, 0) + 1

4


100%|███████████████████████████████████████████| 20/20 [00:52<00:00,  2.62s/it]


In [10]:
# Strategy 1 with betweenness

lower_bound = 20
upper_bound = 40
nb_of_iters = 4
solutions_prob_ms_bc_add = {}

G = G_fb
nodes_to_add = []
nb_of_nodes_in_resolving_set = 20

length = dict(nx.all_pairs_shortest_path_length(G))

c = nx.betweenness_centrality(G)

# normalize the betweenness centralities
c_values = np.array(list(c.values()))
c_norm = c_values / sum(c_values)
node_list = list(c.keys())

while True:
    intensities = {str(key): 0 for key in G.nodes}
    for i in range(100):
        # Random set of nodes to test
        nodes = set(np.random.choice(node_list, p=c_norm, size=nb_of_nodes_in_resolving_set, replace=False))
        resolved = set_resolved(G, nodes, length)
        for node in resolved:
            intensities[node] += +1

    small = [key for key, val in intensities.items() if val < 40]
    s = len(small)
    nodes_to_add += get_nodes_with_diff_neighbors(G, small)
    if s == 0:
        break
indices = [node_list.index(i) for i in nodes_to_add] 
node_list = [element for index, element in enumerate(node_list) if index not in indices]
c_norm = [element for index, element in enumerate(c_norm) if index not in indices]
c_norm = c_norm / sum(c_norm)

for nb in range(0, len(nodes_to_add)):
    solutions_prob_ms_bc_add[nb] = solutions_prob_ms_bc_add.get(nb, 0) + 0  
for nb in tqdm(range(len(nodes_to_add), upper_bound, 1)): # G.number_of_nodes()
    num_nodes = nb - len(nodes_to_add) # Number of nodes to sample
    count = 0
    for i in range(nb_of_iters):
        nodes = nodes_to_add + list(np.random.choice(node_list, p=c_norm, size=num_nodes, replace=False)) # Random set of nodes to test
        if is_resolving_set(G, nodes, length):
            count += 1
    solutions_prob_ms_bc_add[nb] =  solutions_prob_ms_bc_add.get(nb, 0) + (count / nb_of_iters)
for nb in range(upper_bound, G_fb.number_of_nodes()):
        solutions_prob_ms_bc_add[nb] = solutions_prob_ms_bc_add.get(nb, 0) + 1

KeyboardInterrupt: 

In [253]:
#with open('simulations/fb_transition_eigen_400iters_10step.pickle', 'wb') as file:
    #pickle.dump(solutions_prob_eigen_fb, file, protocol=pickle.HIGHEST_PROTOCOL)

In [38]:
# Define your data
x = list(solutions_rand_fb.keys())
y = list(solutions_rand_fb.values())


x2 = list(solutions_prob_d.keys())
y2 = list(solutions_prob_d.values())

x3 = list(solutions_prob_bc_fb.keys())
y3 = list(solutions_prob_bc_fb.values())

x4 = list(solutions_prob_dc_fb.keys())
y4 = list(solutions_prob_dc_fb.values())

x5 = list(solutions_prob_so_fb.keys())
y5 = list(solutions_prob_so_fb.values())

x6 = list(solutions_prob_cc_fb.keys())
y6 = list(solutions_prob_cc_fb.values())

x7 = list(solutions_prob_ms_bc_add.keys())
y7 = list(solutions_prob_ms_bc_add.values())

x8 = list(solutions_prob_eigen_fb.keys())
y8 = list(solutions_prob_eigen_fb.values())

x9 = list(solutions_prob_ms_d.keys())
y9 = list(solutions_prob_ms_d.values())

x10 = list(solutions_prob_ms_bc.keys())
y10 = list(solutions_prob_ms_bc.values())

# Define the trace for the scatter plot
trace = go.Scatter(x=x, y=y, mode='markers+lines', name='Random vertices')
trace2 = go.Scatter(x=x2, y=y2, mode='markers+lines', name='Random vertices (prop to degree)')
trace3 = go.Scatter(x=x3, y=y3, mode='markers+lines', name='Betweeness centrality')
trace4 = go.Scatter(x=x4, y=y4, mode='markers+lines', name='Degree centrality')
trace5 = go.Scatter(x=x5, y=y5, mode='markers+lines', name='Second order centrality)')
trace6 = go.Scatter(x=x6, y=y6, mode='markers+lines', name='Closness centrality')
trace7 = go.Scatter(x=x7, y=y7, mode='markers+lines', name='Magic sauce (betweenness + trick)')
trace8 = go.Scatter(x=x8, y=y8, mode='markers+lines', name='Eigenvector centrality')
trace9 = go.Scatter(x=x9, y=y9, mode='markers+lines', name='Magic Sauce (degree)')
trace10 = go.Scatter(x=x10, y=y10, mode='markers+lines', name='Magic sauce (betweenness)')

# Define the layout
layout = go.Layout(#title='Probability of resolving the graph as a function of the subset cardinality', 
                   #title_x=0.5,
                   xaxis=dict(title='Cardinality of the subset'), 
                   yaxis=dict(title='Probability of resolving the graph'))
                   #legend=dict(x=0.67, y=0.08, orientation='v'))

# Combine the traces and layout into a figure
fig = go.Figure(data=[trace, trace1, trace3, trace4, trace5, trace6, trace7, trace8, trace9, trace10], layout=layout)

# Show the figure
fig.show()

In [254]:
def sim_centrality_box_real(G, centrality_f, nb_graph=1, nb_of_iters=400):
    
    box_sol = []
    
    # Generate the random graph and compute shortest paths
    G = G_fb
    length = dict(nx.all_pairs_shortest_path_length(G))

    c = centrality_f(G)

    # normalize the centralities
    c_values = np.array(list(c.values()))
    c_norm = c_values / sum(c_values)
    node_list = list(c.keys())

    for _ in range(nb_of_iters):
        nodes = np.random.choice(node_list, p=c_norm, size=len(G), replace=False) # Random set of nodes to test
        for nb in range(0, len(G)):
            if is_resolving_set(G, nodes[:nb], length):
                box_sol.append(nb)
                break
                    
    return box_sol

In [170]:
# Betweenness

betweenness_box_sol = []

nb_of_iters = 100

# Generate the random graph and compute shortest paths
G = G_fb
length = dict(nx.all_pairs_shortest_path_length(G))

c = nx.betweenness_centrality(G, endpoints=True)

# normalize the centralities
c_values = np.array(list(c.values()))
c_norm = c_values / sum(c_values)
node_list = list(c.keys())

for _ in tqdm(range(nb_of_iters)):
    nodes = np.random.choice(node_list, p=c_norm, size=len(G), replace=False) # Random set of nodes to test
    for nb in range(0, len(G)):
        if is_resolving_set(G, nodes[:nb], length):
            betweenness_box_sol.append(nb)
            break

  1%|▍                                        | 1/100 [01:38<2:42:46, 98.65s/it]

653


  2%|▊                                        | 2/100 [02:24<1:49:58, 67.33s/it]

415


  3%|█▏                                       | 3/100 [03:05<1:29:24, 55.31s/it]

...

276


 99%|███████████████████████████████████████▌| 99/100 [1:23:03<00:45, 45.03s/it]

433


100%|███████████████████████████████████████| 100/100 [1:23:05<00:00, 49.85s/it]

80


In [169]:
# Degree

degree_box_sol = []

nb_of_iters = 100

# Generate the random graph and compute shortest paths
G = G_fb
length = dict(nx.all_pairs_shortest_path_length(G))

c = nx.degree_centrality(G)

# normalize the centralities
c_values = np.array(list(c.values()))
c_norm = c_values / sum(c_values)
node_list = list(c.keys())

for _ in range(nb_of_iters):
    nodes = np.random.choice(node_list, p=c_norm, size=len(G), replace=False) # Random set of nodes to test
    for nb in range(0, len(G)):
        if is_resolving_set(G, nodes[:nb], length):
            degree_box_sol.append(nb)
            break

754
658
655
783
583
666
619
726
767
610
697
704
329
775
476
517
496
385
727
670
763
786
381
664
606
777
713
563
399
758
712
770
728
770
455
688
778
657
650
574
665
350
716
746
707
768
759
684
692
611
767
738
779
686
422
787
714
711
628
463
775
783
700
755
774
707
772
742
722
771
593
722
702
795
774
687
746
765
753
342
695
789
710
320
461
673
708
684
763
668
671
524
786
582
582
793
676
766
729
729


In [179]:
# Random

nb_of_iters = 100
rand_box_real = []

# Generate the random graph and compute shortest paths
G = G_fb
length = dict(nx.all_pairs_shortest_path_length(G))

for _ in tqdm(range(nb_of_iters)):
    nodes = list(random.sample(list(G.nodes), len(G)))
    for nb in range(0, len(G)):
        if is_resolving_set(G, nodes[:nb], length):
            rand_box_real.append(nb)
            print(nb)
            break

  1%|▍                                          | 1/100 [00:07<12:08,  7.36s/it]

316


  2%|▊                                          | 2/100 [00:39<36:13, 22.18s/it]

704


  3%|█▎                                         | 3/100 [00:40<20:11, 12.49s/it]

...

266


 99%|█████████████████████████████████████████▌| 99/100 [19:06<00:11, 11.87s/it]

514


100%|█████████████████████████████████████████| 100/100 [19:18<00:00, 11.59s/it]

426


In [199]:
# Strategy 1 with degree

start1_degree_box_real = []

tresh = 50
nb_of_iters = 100

G = G_fb
nb_of_nodes_in_resolving_set = 20

length = dict(nx.all_pairs_shortest_path_length(G))
c = nx.degree_centrality(G)

# normalize the betweenness centralities
c_values = np.array(list(c.values()))
c_norm = c_values / sum(c_values)
node_list = list(c.keys())
ms_b_box_sol_real = []
nodes_to_add = []

while True:
    intensities = {str(key): 0 for key in G.nodes}
    for i in range(100):
        # Random set of nodes to test
        nodes = nodes_to_add + list(np.random.choice(node_list, p=c_norm, size=nb_of_nodes_in_resolving_set, replace=False))
        resolved = set_resolved(G, nodes, length)
        for node in resolved:
            intensities[node] += +1

    small = [key for key, val in intensities.items() if val < tresh]
    s = len(small)
    nodes_to_add += get_nodes_with_diff_neighbors(G, small)
    if s == 0:
        break
indices = [node_list.index(i) for i in nodes_to_add] 
node_list_without_indices_to_add = [element for index, element in enumerate(node_list) if index not in indices]
c_norm_without_indices_to_add = [element for index, element in enumerate(c_norm) if index not in indices]
c_norm_without_indices_to_add = c_norm_without_indices_to_add / sum(c_norm_without_indices_to_add)

for _ in tqdm(range(nb_of_iters)):
    nodes = nodes_to_add + list(np.random.choice(node_list_without_indices_to_add, p=c_norm_without_indices_to_add, size=len(node_list_without_indices_to_add), replace=False)) # Random set of nodes to test
    for nb in range(len(nodes_to_add), len(G)):
        if is_resolving_set(G, nodes[:nb], length):
            start1_degree_box_real.append(nb)
            break

  2%|▊                                          | 2/100 [00:00<00:18,  5.28it/s]

48
35


  3%|█▎                                         | 3/100 [00:00<00:29,  3.26it/s]

58


  5%|██▏                                        | 5/100 [00:01<00:29,  3.17it/s]

...

38
44


 98%|█████████████████████████████████████████▏| 98/100 [00:32<00:00,  3.85it/s]

52


100%|█████████████████████████████████████████| 100/100 [00:34<00:00,  2.93it/s]

128
36


In [173]:
# Strategy 1 with betweenness

start1_betweenness_box_real = []

tresh = 50
nb_of_iters = 100

G = G_fb
nb_of_nodes_in_resolving_set = 20

length = dict(nx.all_pairs_shortest_path_length(G))
c = nx.betweenness_centrality(G, endpoints=True)

# normalize the betweenness centralities
c_values = np.array(list(c.values()))
c_norm = c_values / sum(c_values)
node_list = list(c.keys())
ms_b_box_sol_real = []
nodes_to_add = []

while True:
    intensities = {str(key): 0 for key in G.nodes}
    for i in range(100):
        # Random set of nodes to test
        nodes = nodes_to_add + list(np.random.choice(node_list, p=c_norm, size=nb_of_nodes_in_resolving_set, replace=False))
        resolved = set_resolved(G, nodes, length)
        for node in resolved:
            intensities[node] += +1

    small = [key for key, val in intensities.items() if val < tresh]
    s = len(small)
    nodes_to_add += get_nodes_with_diff_neighbors(G, small)
    if s == 0:
        break
indices = [node_list.index(i) for i in nodes_to_add] 
node_list_without_indices_to_add = [element for index, element in enumerate(node_list) if index not in indices]
c_norm_without_indices_to_add = [element for index, element in enumerate(c_norm) if index not in indices]
c_norm_without_indices_to_add = c_norm_without_indices_to_add / sum(c_norm_without_indices_to_add)

for _ in tqdm(range(nb_of_iters)):

    #print(len(node_list_without_indices_to_add))
    nodes = nodes_to_add + list(np.random.choice(node_list_without_indices_to_add, p=c_norm_without_indices_to_add, size=len(node_list_without_indices_to_add), replace=False)) # Random set of nodes to test
    for nb in range(len(nodes_to_add), len(G)):
        if is_resolving_set(G, nodes[:nb], length):
            start1_betweenness_box_real.append(nb)
            break

  1%|▍                                          | 1/100 [00:00<01:33,  1.06it/s]

93


  2%|▊                                          | 2/100 [00:01<00:51,  1.91it/s]

46


  3%|█▎                                         | 3/100 [00:01<00:44,  2.19it/s]

...

51
37


 99%|█████████████████████████████████████████▌| 99/100 [00:32<00:00,  2.83it/s]

66
32


100%|█████████████████████████████████████████| 100/100 [00:33<00:00,  2.97it/s]

104


In [ ]:
# Strategy 2
start2_box_real = []

nb_of_iters = 100

G = G_fb
nb_of_nodes_in_resolving_set = 20

length = dict(nx.all_pairs_shortest_path_length(G))

for _ in range(nb_of_iters):
    nodes_to_add = []

    while True:
        intensities = {str(key): 0 for key in G.nodes}
        for i in range(nb_of_iters):
            # Random set of nodes to test
            nodes = nodes_to_add + list(np.random.choice(G.nodes, size=nb_of_nodes_in_resolving_set, replace=False))
            resolved = set_resolved(G, nodes, length)
            for node in resolved:
                intensities[node] += 1

        hardest_node_to_resolve = min(intensities, key=intensities.get)
        nodes_to_add.append(hardest_node_to_resolve)
        if is_resolving_set(G, nodes_to_add, length):
            start2_box_real.append(len(nodes_to_add))
            break

In [200]:
dict_result_real = {'Strategy 1 degree':start1_degree_box_real, 
                    'Strategy 1 betweenness':start1_betweenness_box_real, 
                    'Strategy 2':start2_box_real, 
                    'Random strategy':rand_box_real,
                    'Betweenness strategy':betweenness_box_sol,
                    'Degree strategy':degree_box_sol
                   }

In [205]:
fig = go.Figure()

for group, values in dict_result_real.items():
    fig.add_trace(go.Box(y=values, name=group, boxpoints='all'))

    
fig.update_layout(
    yaxis=dict(
        title="Size of the resolving set",
        titlefont=dict(size=12, color='black')
    )
)
fig.show()

In [53]:
# Here we check if the values of the threshold have a big influence

dict_tresh = {}

tresh_values = [25]
nb_graph = 1
nb_of_iters = 100

G = G_calls
print(len(G))
nb_of_nodes_in_resolving_set = 20

length = dict(nx.all_pairs_shortest_path_length(G))

c = nx.degree_centrality(G)

# normalize the betweenness centralities
c_values = np.array(list(c.values()))
c_norm = c_values / sum(c_values)
node_list = list(c.keys())
for tresh in tresh_values:
    ms_b_box_sol_real = []
    nodes_to_add = []

    while True:
        intensities = {str(key): 0 for key in G.nodes}
        for i in range(100):
            # Random set of nodes to test
            nodes = nodes_to_add + list(np.random.choice(node_list, p=c_norm, size=nb_of_nodes_in_resolving_set, replace=False))
            resolved = set_resolved(G, nodes, length)
            for node in resolved:
                intensities[node] += +1

        small = [key for key, val in intensities.items() if val < tresh]
        s = len(small)
        nodes_to_add += get_nodes_with_diff_neighbors(G, small)
        if s == 0:
            print("We manually add {} nodes".format(len(nodes_to_add)))
            break
    indices = [node_list.index(i) for i in nodes_to_add] 
    node_list_without_indices_to_add = [element for index, element in enumerate(node_list) if index not in indices]
    c_norm_without_indices_to_add = [element for index, element in enumerate(c_norm) if index not in indices]
    c_norm_without_indices_to_add = c_norm_without_indices_to_add / sum(c_norm_without_indices_to_add)
    for _ in tqdm(range(nb_of_iters)):
        
        nodes = nodes_to_add + list(np.random.choice(node_list_without_indices_to_add, p=c_norm_without_indices_to_add, size=len(node_list_without_indices_to_add), replace=False)) # Random set of nodes to test
        for nb in range(len(nodes_to_add), len(G)):
            if is_resolving_set(G, nodes[:nb], length):
                ms_b_box_sol_real.append(nb)
                break
    dict_tresh[tresh] = ms_b_box_sol_real

347
We manually add 62 nodes


100%|████████████████████████████████████████| 100/100 [00:00<00:00, 191.79it/s]


In [54]:
fig = go.Figure()

for group, values in dict_tresh.items():
    fig.add_trace(go.Box(y=values, name=group))

fig.show()

## Copenhagen calls graph

In [31]:
# read the file and create a graph
G_calls = nx.read_edgelist('../Real graphs simulations/Copenhagen graphs/calls.csv/edges.csv', delimiter=',', data=(('timestamp', int),('duration', int)))

# print the number of nodes and edges
print('Number of nodes:', G_calls.number_of_nodes())
print('Number of edges:', G_calls.number_of_edges())

Number of nodes: 536
Number of edges: 621


In [32]:
nx.is_connected(G_calls)

False

In [33]:
# Remove the small components such that the graph becomes connected
connected_components = nx.connected_components(G_calls)
biggest = max(connected_components, key=len)
G_calls = G_calls.subgraph(biggest)
nx.is_connected(G_calls)

True

In [34]:
len(G_calls)

347

In [35]:
# Betweenness

betweenness_box_sol_calls = []

nb_of_iters = 100

# Generate the random graph and compute shortest paths
G = G_calls
length = dict(nx.all_pairs_shortest_path_length(G))

c = nx.betweenness_centrality(G, endpoints=True)

# normalize the centralities
c_values = np.array(list(c.values()))
c_norm = c_values / sum(c_values)
node_list = list(c.keys())

for _ in tqdm(range(nb_of_iters)):
    nodes = np.random.choice(node_list, p=c_norm, size=len(G), replace=False) # Random set of nodes to test
    for nb in range(0, len(G)):
        if is_resolving_set(G, nodes[:nb], length):
            betweenness_box_sol_calls.append(nb)
            break

100%|█████████████████████████████████████████| 100/100 [14:09<00:00,  8.49s/it]


In [36]:
# Degree

degree_box_sol_calls = []

nb_of_iters = 100

# Generate the random graph and compute shortest paths
G = G_calls
length = dict(nx.all_pairs_shortest_path_length(G))

c = nx.degree_centrality(G)

# normalize the centralities
c_values = np.array(list(c.values()))
c_norm = c_values / sum(c_values)
node_list = list(c.keys())

for _ in range(nb_of_iters):
    nodes = np.random.choice(node_list, p=c_norm, size=len(G), replace=False) # Random set of nodes to test
    for nb in range(0, len(G)):
        if is_resolving_set(G, nodes[:nb], length):
            degree_box_sol_calls.append(nb)
            break

In [37]:
# Random

nb_of_iters = 100
rand_box_real_calls = []

# Generate the random graph and compute shortest paths
G = G_calls
length = dict(nx.all_pairs_shortest_path_length(G))

for _ in tqdm(range(nb_of_iters)):
    nodes = list(random.sample(list(G.nodes), len(G)))
    for nb in range(0, len(G)):
        if is_resolving_set(G, nodes[:nb], length):
            rand_box_real_calls.append(nb)
            break

100%|█████████████████████████████████████████| 100/100 [04:35<00:00,  2.75s/it]


In [51]:
# Strategy 1 with degree

start1_degree_box_real_calls = []

tresh = 25
nb_of_iters = 100

G = G_calls
nb_of_nodes_in_resolving_set = 20

length = dict(nx.all_pairs_shortest_path_length(G))
c = nx.degree_centrality(G)

# normalize the betweenness centralities
c_values = np.array(list(c.values()))
c_norm = c_values / sum(c_values)
node_list = list(c.keys())
ms_b_box_sol_real = []
nodes_to_add = []

while True:
    intensities = {str(key): 0 for key in G.nodes}
    for i in range(100):
        # Random set of nodes to test
        nodes = nodes_to_add + list(np.random.choice(node_list, p=c_norm, size=nb_of_nodes_in_resolving_set, replace=False))
        resolved = set_resolved(G, nodes, length)
        for node in resolved:
            intensities[node] += +1

    small = [key for key, val in intensities.items() if val < tresh]
    s = len(small)
    nodes_to_add += get_nodes_with_diff_neighbors(G, small)

    if s == 0:
        print("We manually add {} nodes".format(len(nodes_to_add)))
        break
indices = [node_list.index(i) for i in nodes_to_add] 
node_list_without_indices_to_add = [element for index, element in enumerate(node_list) if index not in indices]
c_norm_without_indices_to_add = [element for index, element in enumerate(c_norm) if index not in indices]
c_norm_without_indices_to_add = c_norm_without_indices_to_add / sum(c_norm_without_indices_to_add)

for _ in tqdm(range(nb_of_iters)):
    nodes = nodes_to_add + list(np.random.choice(node_list_without_indices_to_add, p=c_norm_without_indices_to_add, size=len(node_list_without_indices_to_add), replace=False)) # Random set of nodes to test
    for nb in range(len(nodes_to_add), len(G)):
        if is_resolving_set(G, nodes[:nb], length):
            start1_degree_box_real_calls.append(nb)
            break

49
61
61


100%|█████████████████████████████████████████| 100/100 [00:20<00:00,  5.00it/s]


In [64]:
# Strategy 1 with betweenness

start1_betweenness_box_real_calls = []

tresh = 15
nb_of_iters = 100

G = G_calls
nb_of_nodes_in_resolving_set = 20

length = dict(nx.all_pairs_shortest_path_length(G))
c = nx.betweenness_centrality(G, endpoints=True)

# normalize the betweenness centralities
c_values = np.array(list(c.values()))
c_norm = c_values / sum(c_values)
node_list = list(c.keys())
ms_b_box_sol_real = []
nodes_to_add = []

while True:
    intensities = {str(key): 0 for key in G.nodes}
    for i in range(100):
        # Random set of nodes to test
        nodes = nodes_to_add + list(np.random.choice(node_list, p=c_norm, size=nb_of_nodes_in_resolving_set, replace=False))
        resolved = set_resolved(G, nodes, length)
        for node in resolved:
            intensities[node] += +1

    small = [key for key, val in intensities.items() if val < tresh]
    s = len(small)
    nodes_to_add += get_nodes_with_diff_neighbors(G, small)
    if s == 0:
        print("We manually add {} nodes".format(len(nodes_to_add)))
        break
indices = [node_list.index(i) for i in nodes_to_add] 
node_list_without_indices_to_add = [element for index, element in enumerate(node_list) if index not in indices]
c_norm_without_indices_to_add = [element for index, element in enumerate(c_norm) if index not in indices]
c_norm_without_indices_to_add = c_norm_without_indices_to_add / sum(c_norm_without_indices_to_add)

for _ in tqdm(range(nb_of_iters)):

    #print(len(node_list_without_indices_to_add))
    nodes = nodes_to_add + list(np.random.choice(node_list_without_indices_to_add, p=c_norm_without_indices_to_add, size=len(node_list_without_indices_to_add), replace=False)) # Random set of nodes to test
    for nb in range(len(nodes_to_add), len(G)):
        if is_resolving_set(G, nodes[:nb], length):
            start1_betweenness_box_real_calls.append(nb)
            break

We manually add 61 nodes


100%|█████████████████████████████████████████| 100/100 [01:04<00:00,  1.56it/s]


In [40]:
# Strategy 2
start2_box_real_calls = []

nb_of_iters = 100

G = G_calls
nb_of_nodes_in_resolving_set = 20

length = dict(nx.all_pairs_shortest_path_length(G))

for _ in range(nb_of_iters):
    nodes_to_add = []

    while True:
        intensities = {str(key): 0 for key in G.nodes}
        for i in range(nb_of_iters):
            # Random set of nodes to test
            nodes = nodes_to_add + list(np.random.choice(G.nodes, size=nb_of_nodes_in_resolving_set, replace=False))
            resolved = set_resolved(G, nodes, length)
            for node in resolved:
                intensities[node] += 1

        hardest_node_to_resolve = min(intensities, key=intensities.get)
        nodes_to_add.append(hardest_node_to_resolve)
        if is_resolving_set(G, nodes_to_add, length):
            start2_box_real_calls.append(len(nodes_to_add))
            break

In [65]:
dict_result_real = {'Strategy 1 degree':start1_degree_box_real_calls, 
                    'Strategy 1 betweenness':start1_betweenness_box_real_calls, 
                    'Strategy 2':start2_box_real_calls, 
                    'Random strategy':rand_box_real_calls,
                    'Betweenness strategy':betweenness_box_sol_calls,
                    'Degree strategy':degree_box_sol_calls
                   }

In [66]:
fig = go.Figure()

for group, values in dict_result_real.items():
    fig.add_trace(go.Box(y=values, name=group, boxpoints='all'))

    
fig.update_layout(
    yaxis=dict(
        title="Size of the resolving set",
        titlefont=dict(size=12, color='black')
    )
)
fig.show()

In [69]:
len(G_fb)

800